# Face mask detection using VGG16 model

In [ ]:
import pandas as pd
import numpy as np
import os

In [2]:
import cv2

In [3]:
category=["WithMask","WithoutMask"]

In [4]:
data=[]
for i in category:
    path=os.path.join('train',i)
    label=category.index(i)
    for i in os.listdir(path):
        img_path=os.path.join(path,i)
        img=cv2.imread(img_path)
        img=cv2.resize(img,(224,224))
        data.append([img,label])

In [5]:
import random 
random.shuffle(data)

In [6]:
x=[]
y=[]
for feature,label in data:
    x.append(feature)
    y.append(label)

In [7]:
len(y)

2196

In [8]:
x=np.array(x)
y=np.array(y)

In [9]:
x=x/255

In [10]:
from sklearn.model_selection import train_test_split
x_train,x_text,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [11]:
from keras.applications.vgg16 import VGG16

In [12]:
vgg=VGG16()

In [13]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [14]:
from keras import Sequential
model =Sequential()

In [15]:
for layer in vgg.layers[:-1]:
    model.add(layer)

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [17]:
for layer in model.layers:
    layer.trainable=False

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [19]:
from keras.layers import Dense
model.add(Dense(1,activation='sigmoid'))

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [21]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [22]:
model.fit(x_train,y_train,epochs=5,validation_data=(x_text,y_test))

Epoch 1/5
55/55 [==============================] - 1204s 22s/step - loss: 0.4950 - accuracy: 0.7597 - val_loss: 0.4404 - val_accuracy: 0.7818
Epoch 2/5
55/55 [==============================] - 1173s 21s/step - loss: 0.3350 - accuracy: 0.8713 - val_loss: 0.2890 - val_accuracy: 0.9023
Epoch 3/5
55/55 [==============================] - 1169s 21s/step - loss: 0.2587 - accuracy: 0.9117 - val_loss: 0.2555 - val_accuracy: 0.9136
Epoch 4/5
55/55 [==============================] - 1168s 21s/step - loss: 0.2252 - accuracy: 0.9226 - val_loss: 0.2275 - val_accuracy: 0.9273
Epoch 5/5
55/55 [==============================] - 1177s 21s/step - loss: 0.2033 - accuracy: 0.9356 - val_loss: 0.2042 - val_accuracy: 0.9341


In [25]:
cap=cv2.VideoCapture(0)

In [47]:
while True:
    ret,frame=cap.read()
    frame=cv2.resize(frame,(224,224))
    y_pred = detect_face_mask(frame)
    if y_pred ==0:
         draw_label(frame,"mask",(30,30),(0,255,0))
    else:
        draw_label(frame,"no mask",(30,30),(0,0,255))
    print(y_pred)
    cv2.imshow('window',frame)
    if cv2.waitKey(1) & 0xFF == ord('x'):
        break
cv2.destroyAllwindows()

1/1 [==============================] - 0s 499ms/step
0
1/1 [==============================] - 1s 534ms/step
0
1/1 [==============================] - 1s 655ms/step
0
1/1 [==============================] - 1s 682ms/step
0
1/1 [==============================] - 1s 631ms/step
0
1/1 [==============================] - 1s 711ms/step
0
1/1 [==============================] - 1s 702ms/step
0
1/1 [==============================] - 1s 664ms/step
0
1/1 [==============================] - 1s 628ms/step
0
1/1 [==============================] - 1s 618ms/step
0
1/1 [==============================] - 1s 618ms/step
0
1/1 [==============================] - 1s 624ms/step
0
1/1 [==============================] - 1s 620ms/step
0
1/1 [==============================] - 1s 650ms/step
0
1/1 [==============================] - 1s 650ms/step
0
1/1 [==============================] - 1s 644ms/step
0
1/1 [==============================] - 1s 634ms/step
0
1/1 [==============================] - 1s 631ms/step
0
1/1 [=====

KeyboardInterrupt: 

In [35]:
def detect_face_mask(img):
    y_pred=model.predict_classes(img.reshape(1,224,224,3))
    return y_pred[0][0]

In [37]:
def detect_face_mask(img):
    y_pred = model.predict(img.reshape(1,224,224,3))
    predicted_class = y_pred.argmax(axis=-1)
    return predicted_class[0]

In [46]:
def draw_label(img,text,pos,bg_color):
    text_size=cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)
    end_x=pos[0] + text_size[0][0] +2
    end_y=pos[1] + text_size[0][1] -2
    
    cv2.rectangle(img,pos,(end_x,end_y),bg_color,cv2.FILLED)
    cv2.putText(img,text,pos,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),1,cv2.LINE_AA)

# Face mask detection using ResNet50 model

In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import Flatten, Dense

# Define categories
cat = ['with_mask', 'without_mask']

# Load and preprocess images
data = []
for i in cat:
    temp = 'Face_mask_data'
    label = cat.index(i)
    path = os.path.join(temp, i)
    for file_name in os.listdir(path):
        img_path = os.path.join(path, file_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))  # Resize image to (224, 224)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        data.append([img, label])

# Shuffle data
np.random.shuffle(data)

# Split data into features and labels
x = np.array([item[0] for item in data])
y = np.array([item[1] for item in data])

# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

# Load pre-trained ResNet50 model
res = ResNet50(weights='imagenet', include_top=False,input_shape=(224, 224, 3))


for layer in res.layers:
    layer.trainable = False

# Create a new Sequential model
model = Sequential()

# Add the pre-trained ResNet50 model to the new model
model.add(res)
from keras.layers import Dense,Flatten
model.add(Flatten())
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=2,validation_data=(x_test,y_test))

Epoch 1/2
55/55 [==============================] - 510s 9s/step - loss: 0.0822 - accuracy: 0.9834 - val_loss: 0.0798 - val_accuracy: 0.9973
Epoch 2/2
55/55 [==============================] - 495s 9s/step - loss: 0.0020 - accuracy: 0.9994 - val_loss: 0.0105 - val_accuracy: 0.9987


In [ ]:
cap=cv2.VideoCapture(0)

In [ ]:
while True:
    ret,frame=cap.read()
    frame=cv2.resize(frame,(224,224))
    y_pred = detect_face_mask(frame)
    if y_pred ==1:
         draw_label(frame,"no mask",(30,30),(0,0,255))
    else:
        draw_label(frame,"mask",(30,30),(0,255,0))
    print(y_pred)
    cv2.imshow('window',frame)
    if cv2.waitKey(1) & 0xFF == ord('x'):
        break
cv2.destroyAllwindows()

In [ ]:
def detect_face_mask(img):
    y_pred = model.predict(img.reshape(1,224,224,3))
    predicted_class = y_pred.argmax(axis=-1)
    return predicted_class

In [ ]:
def draw_label(img,text,pos,bg_color):
    text_size=cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)
    end_x=pos[0] + text_size[0][0] +2
    end_y=pos[1] + text_size[0][1] -2
    
    cv2.rectangle(img,pos,(end_x,end_y),bg_color,cv2.FILLED)
    cv2.putText(img,text,pos,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),1,cv2.LINE_AA)